

# Deflection from Radial Trajectory with a Disk Potential

We have a star ejected from the Galactic Center at $t = 0$ with:

- Initial speed $v_{\mathrm{ej}}$.
- Polar angle $\theta$ relative to the Galactic plane ($\theta = 0^\circ$ is purely in-plane, $\theta = 90^\circ$ is perpendicular to the plane).

The total Galactic potential is:

$$
\Phi_{\mathrm{tot}}(r,R,z) = -\frac{GM}{r} + \Phi_{\mathrm{MN}}(R,z),
$$

where:

- $\Phi_0(r) = -\frac{GM}{r}$ is a Keplerian potential with mass $M$.  
- $\Phi_{\mathrm{MN}}(R,z)$ is the Miyamoto–Nagai disk potential, treated as a perturbation to radial motion.

---

### Miyamoto–Nagai Disk Potential

The Miyamoto–Nagai disk potential is given by:

$$
\Phi_{\mathrm{MN}}(R,z) = -\frac{G M_d}{\sqrt{R^2 + \bigl(a + \sqrt{z^2 + b^2}\bigr)^2}},
$$

where $M_d$ is the disk mass, and $a, b$ are scale lengths. The force components in cylindrical coordinates $(R,z)$ are:

$$
F_R = \frac{G M_d R}{\bigl[R^2 + (a + \sqrt{z^2 + b^2})^2\bigr]^{3/2}},
$$

$$
F_z = \frac{G M_d \bigl(a + \sqrt{z^2 + b^2}\bigr) z}{\bigl[R^2 + (a + \sqrt{z^2 + b^2})^2\bigr]^{3/2} \sqrt{z^2 + b^2}}.
$$

---

### Zeroth-Order Radial Orbit in Kepler Potential

Ignoring the disk potential, the radial motion in a Kepler potential is governed by energy conservation:

$$
E = \frac{1}{2} \dot{r}^2 - \frac{GM}{r}.
$$

For a star launched outward from $r = 0$ with speed $v_{\mathrm{ej}}$, the total energy is:

$$
E = \frac{1}{2} v_{\mathrm{ej}}^2.
$$

The radial speed at a later radius $r$ is:

$$
\dot{r}_0(r) = \sqrt{v_{\mathrm{ej}}^2 + \frac{2GM}{r}}.
$$

The time–radius relation is:

$$
t(r) = \int_{0}^{r} \frac{dr'}{\sqrt{v_{\mathrm{ej}}^2 + \frac{2GM}{r'}}}.
$$

---

### Angular Deviation Due to the Disk Potential

The angular deviation $\delta \alpha$ is induced by the disk torque. The transverse force component is:

$$
F_{\perp}(r,\theta) = -\sin\theta F_R + \cos\theta F_z,
$$

where $F_R$ and $F_z$ are evaluated at $(R, z) = (r \cos\theta, r \sin\theta)$.

The transverse velocity kick is:

$$
\delta v_{\perp} = \int_{0}^{r_f} \frac{F_{\perp}(r, \theta)}{\dot{r}_0(r)} \, dr,
$$

and the angular deviation is:

$$
\delta \alpha(\theta, v_{\mathrm{ej}}) = \frac{\delta v_{\perp}}{\dot{r}_0(r_f)}.
$$

---

### Dimensionless Form

For large $v_{\mathrm{ej}}$, the dimensionless combination:

$$
\chi = \frac{G M_d}{b v_{\mathrm{ej}}^2},
$$

determines the scaling of $\kappa$. The deflection $\kappa$ decreases as $v_{\mathrm{ej}}^{-2}$.

---

### From Galactocentric $\delta \alpha$ to Heliocentric Distance Variation

The Galactocentric angular deviation $\delta \alpha$ translates into a change in the inferred heliocentric distance $D_{\mathrm{implied}}$ versus the true heliocentric distance $D_{\mathrm{true}}$. The difference is approximately:

$$
D_{\mathrm{implied}} - D_{\mathrm{true}} \approx R_0 \, \delta \alpha \, \sin\phi,
$$

where $R_0 \approx 8\,\text{kpc}$ is the Sun's Galactocentric distance, and $\phi$ is the angle between the Sun–GC vector and the star's radial trajectory.



In [1]:
import numpy as np
from scipy.integrate import quad

def delta_alpha(v_ej, theta, M, M_d, a, b, r_f, G=4.3009e-3):
    """
    Compute the galactocentric angular deviation delta_alpha given v_ej and theta.
    
    Parameters
    ----------
    v_ej : float
        Ejection speed from the Galactic Center (in suitable units, e.g. km/s).
    theta : float
        Polar angle in radians of the ejection direction relative to the Galactic plane.
    M : float
        Point mass (bulge) mass parameter for the Keplerian potential (in suitable units).
    M_d : float
        Miyamoto-Nagai disk mass (in same units as M).
    a : float
        Miyamoto-Nagai disk scale length a.
    b : float
        Miyamoto-Nagai disk scale height b.
    r_f : float
        Final galactocentric radius at which we measure the star (in suitable distance units).
    G : float, optional
        Gravitational constant in chosen units.
        Default is 4.3009e-3 (km²/s²·pc/Msun) commonly used in galactic dynamics.
    
    Returns
    -------
    delta_alpha : float
        The galactocentric angular deviation in radians.
    """
    
    # Define the radial velocity from energy conservation in Kepler potential:
    # E = v_ej²/2, so dot{r}(r) = sqrt(v_ej² + 2GM/r)
    def radial_speed(r):
        return np.sqrt(v_ej**2 + 2*G*M/r)

    # MN disk forces:
    # R = r cos(theta), z = r sin(theta)
    def F_R(R, z):
        denom = (R**2 + (a + np.sqrt(z**2 + b**2))**2)**1.5
        return G*M_d*R/denom
    
    def F_z(R, z):
        A = a + np.sqrt(z**2 + b**2)
        denom = (R**2 + A**2)**1.5
        return G*M_d*A*z/(denom * np.sqrt(z**2 + b**2))

    # Perp force integrand:
    # F_perp(r,theta) = -sin(theta)*F_R + cos(theta)*F_z at (R=r cos(theta), z=r sin(theta))
    def integrand(r):
        R = r*np.cos(theta)
        z = r*np.sin(theta)
        FR = F_R(R, z)
        FZ = F_z(R, z)
        F_perp = -np.sin(theta)*FR + np.cos(theta)*FZ
        return F_perp / radial_speed(r)

    # Integrate from 0 to r_f:
    # delta_alpha = (1/dot{r}(r_f)) * integral_0^{r_f} [F_perp(r,theta)/dot{r}(r)] dr
    val, _ = quad(integrand, 0, r_f)
    return val / radial_speed(r_f)

# Example usage (with placeholder values for M, M_d, a, b, r_f):
# You must set these parameters appropriately for a given problem.
M = 1e10       # Mass parameter (in solar masses)
M_d = 5e9      # Disk mass
a = 3.0e3      # Scale length (in pc)
b = 0.3e3      # Scale height (in pc)
r_f = 5.0e4    # Final radius (in pc)
theta = np.radians(30)  # 30 degrees above the plane
v_ej = 1000.0  # Ejection speed in km/s

da = delta_alpha(v_ej, theta, M, M_d, a, b, r_f)
print("Delta alpha [radians]:", da)


Delta alpha [radians]: 0.0030205308043711842
